In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import time

In [2]:
driver = webdriver.Chrome()
driver.get('https://airtable.com/app1PaujS9zxVGUZ4/shrqYt5kSqMzHV9R5/tbl8c8kanuNB6bPYr?backgroundColor=green&viewControls=on',)
time.sleep(1)

In [3]:
scrollbar = "//*[@id= 'view']/div/div[1]/div[1]/div[3]"
element = driver.find_element(By.XPATH, scrollbar)


In [4]:
leftPane = []
rightPane = []

for _ in range(240):
    start_time = time.time()  # Start time of the loop

    # Perform the desired action
    ActionChains(driver).click_and_hold(element).move_by_offset(0, 1).perform()
    ActionChains(driver).reset_actions()

    # Get the page source and parse it
    html = driver.page_source
    parsed_html = BeautifulSoup(html, "html.parser")
    
    # Append data to the respective lists
    left_data = parsed_html.find_all('div', class_='dataRow leftPane rowExpansionEnabled rowSelectionEnabled')
    right_data = parsed_html.find_all('div', class_='dataRow rightPane rowExpansionEnabled rowSelectionEnabled')
    leftPane.append(left_data)
    rightPane.append(right_data)
    
    # Calculate elapsed time
    end_time = time.time()  # End time of the loop
    elapsed_time = end_time - start_time  # Time taken for one loop iteration

    # If the operation is faster than 1/5 second, pause for the remaining time to comply with the rate limit
    if elapsed_time < 1/5:
        time.sleep(1/5 - elapsed_time)

In [5]:
# Initialize an empty dictionary to store the matches
matches = {}

# Define the pattern to search for
pattern = r'<div class="line-height-4 overflow-hidden truncate">(.*?)<\/div>'

# Loop over each element in leftPane
for element in leftPane:
    # Loop over each sub-element in the current element
    for sub_element in element:
        # Convert the sub-element to a string and search for the pattern
        match = re.search(pattern, str(sub_element))
        
        # If a match is found, add it to the dictionary
        if match:
            # Use a slice of the sub-element as the key and the matched group as the value
            key = str(sub_element)[70:100]
            value = str(match.group(1))
            
            matches[key] = value

In [6]:
data_dict = {}

# Define patterns
url_pattern = r'<span class="url">(.*?)</span>'
common_pattern = r'<div class="flex-auto truncate-pre" title="(.*?)">'
pattern = r'<div class="flex-auto truncate line-height-4 right-align tabular-nums" style="padding:6px">(.*?)</div>'
date_pattern = r'<div class="truncate css-10jy3hn">(.*?)</div></div></div>'

for i, right_pane_row in enumerate(rightPane):
    for j, right_pane_item in enumerate(right_pane_row):
        right_pane_str = str(right_pane_item)
        key = matches.get(right_pane_str[71:101])
        
        if key:
            html = right_pane_str
            company_name = matches.get(str(leftPane[i][j])[70:100])
            
            data_dict[key] = {
                'company_name': company_name,
                'percentage': '',
                'location': '',
                'url': '',
                'industry': '',
                '#layoff': '',
                'stage': '',
                'date': ''
            }
            
            # Extract URL
            url_match = re.search(url_pattern, html)
            if url_match:
                data_dict[key]['url'] = url_match.group(1)
            
            # Extract common info
            common_match = re.findall(common_pattern, html)
            if common_match:
                data_dict[key]['location'] = common_match[0]
                if len(common_match) == 4:
                    data_dict[key]['country'] = common_match[3]
                    data_dict[key]['stage'] = common_match[2]
                    data_dict[key]['industry'] = common_match[1]
                elif len(common_match) == 3:
                    data_dict[key]['country'] = common_match[2]
                else:
                    data_dict[key]['country'] = common_match[4]
                    data_dict[key]['industry'] = common_match[2]
            
            # Extract other info
            match = re.findall(pattern, right_pane_str)
            if match:
                if len(match) == 3:
                    data_dict[key]['#layoff'] = match[0]
                    data_dict[key]['percentage'] = match[1]
                    data_dict[key]['raised'] = match[2]
                elif len(match) == 1:
                    data_dict[key]['raised'] = match[0]
                elif len(match) == 2:
                    if '%' in match[0]:
                        data_dict[key]['percentage'] = match[0]
                        data_dict[key]['raised'] = match[1]
                    elif '%' in match[1]:
                        data_dict[key]['#layoff'] = match[0]
                        data_dict[key]['percentage'] = match[1]
                    else:
                        data_dict[key]['#layoff'] = match[0]
                        data_dict[key]['raised'] = match[1]
            
            # Extract date
            date_match = re.search(date_pattern, html)
            if date_match:
                data_dict[key]['date'] = date_match.group(1)

print(len(data_dict))


1521


In [7]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index')
df.reset_index(drop=True, inplace=True)
# Assuming your DataFrame is named 'df'
new_column_names = {
    'company_name': 'Company',
    'percentage': 'Percentage',
    'location': 'Location HQ',
    'url': 'URL',
    'industry': 'Industry',
    '#layoff': '# Laid Off',
    'stage': 'Stage',
    'date': 'Date',
    'country': 'Country',
    'raised': '$ Raised (mm)'
}

df.rename(columns=new_column_names, inplace=True)

In [9]:
#Extract data in a csv file
df.to_csv('../data/layoffs_data.csv')